In [19]:
import _threshold_utils as threshold_utils
import altair as alt

import pandas as pd
import geopandas as gpd
from segment_speed_utils.project_vars import analysis_date
from shared_utils import calitp_color_palette as cp

In [20]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [21]:
GCS_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/"

In [22]:
analysis_date

'2023-03-15'

In [23]:
speed_stops_subset = ['gtfs_dataset_key', '_gtfs_dataset_name', 'shape_array_key',
       'stop_sequence', 'trip_id','speed_mph']

In [24]:
speed_stops = pd.read_parquet(f"{GCS_PATH}speeds_stop_segments_{analysis_date}")

In [63]:
speed_stops.sample()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,stop_sequence,trip_id,min_time,min_dist,max_time,max_dist,meters_elapsed,sec_elapsed,speed_mph
__null_dask_index__,,,,,,,,,,,,
1419102,ec4a1466e6836587e9754591dca807d0,Bay Area 511 Muni VehiclePositions,f46717b0a40542eec3dbfe55ac8d8058,14,11042111_M11,2023-03-15 12:17:43,71.11,2023-03-15 12:18:33,81.79,10.68,50.00,0.48


In [25]:
speed_stops2 = speed_stops[speed_stops_subset]

In [29]:
# Do I need geometry? Doesn't seem like it..
avg_speeds = gpd.read_parquet(f"{GCS_PATH}avg_speeds_stop_segments_{analysis_date}.parquet")

In [31]:
avg_speeds_subset = ['geometry','geometry_arrowized','district','district_name']
avg_speeds2 = avg_speeds.drop(columns = avg_speeds_subset)

In [33]:
m1 = pd.merge(avg_speeds2, speed_stops2, on = ['gtfs_dataset_key','shape_array_key', 'stop_sequence'], how = 'inner')

In [34]:
m1.stop_sequence.describe()

count   2793704.00
mean        382.17
std        1018.44
min           0.00
25%          13.00
50%          28.00
75%          54.00
max       12000.00
Name: stop_sequence, dtype: float64

In [ ]:
# one_route3 = m1[m1.shape_array_key == "79546070e205e6085de4597ac1063d77"]

In [ ]:
# one_route3['test_seq'] = one_route3.groupby(['gtfs_dataset_key','shape_array_key', 'trip_id']).cumcount().astype(int)+1

In [ ]:
#one_route3

In [36]:
# Renumber stop sequences since some of them are out of order
m1 = m1.sort_values(['shape_array_key', 'gtfs_dataset_key', 'trip_id', 'stop_sequence']).reset_index(drop = True)

In [37]:
m1['Test Stop Sequence'] = m1.groupby(['gtfs_dataset_key','shape_array_key', 'trip_id']).cumcount().astype(int)+1

In [69]:
foothill3 = speed_stops2[speed_stops2.shape_array_key == "053bf66eba3f5d9980daafd812b99161"]

In [70]:
len(foothill3)

29888

In [71]:
foothill3.head()

,gtfs_dataset_key,_gtfs_dataset_name,shape_array_key,stop_sequence,trip_id,speed_mph
__null_dask_index__,,,,,,
2538062,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,3218,t604-b2791-sl5,4.76
2538063,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,3218,t64A-b275D-sl5,9.90
2538064,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,3218,t4D1-b27B5-sl5,25.60
2538065,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,3218,t802-b2790-sl5,34.24
2538066,7342e9ab4e268b54f9cce8cc25f11345,Foothill Vehicle Positions,053bf66eba3f5d9980daafd812b99161,3218,t613-b2775-sl5,8.07


In [65]:
foothill2 = avg_speeds[avg_speeds.shape_array_key == "053bf66eba3f5d9980daafd812b99161"]

In [66]:
len(foothill2)

752

In [67]:
foothill = m1[m1.shape_array_key == "053bf66eba3f5d9980daafd812b99161"]

In [68]:
len(foothill)

29888

In [53]:
foothill['Test Stop Sequence'].describe()

count   29888.00
mean      288.94
std       167.54
min         1.00
25%       144.00
50%       288.00
75%       432.00
max       640.00
Name: Test Stop Sequence, dtype: float64

In [54]:
foothill['Test Stop Sequence'].value_counts().head()

1      52
271    52
269    52
268    52
267    52
Name: Test Stop Sequence, dtype: int64

In [55]:
foothill['_gtfs_dataset_name'].unique()

array(['Foothill Vehicle Positions'], dtype=object)

In [72]:
foothill.groupby(['trip_id']).agg({'stop_sequence':'count'}).sort_values(['stop_sequence'], ascending = False).head()

,stop_sequence
trip_id,
t604-b2791-sl5,640
t4D1-b27B5-sl5,616
t59B-b2793-sl5,614
t668-b278A-sl5,611
t64A-b275D-sl5,611


In [74]:
foothill[muni['trip_id'] == "t604-b2791-sl5"].head(10)

,shape_array_key,stop_sequence,gtfs_dataset_key,loop_or_inlining,avg_speed_mph,n_trips,p20_speed_mph,p80_speed_mph,_gtfs_dataset_name,trip_id,speed_mph,Test Stop Sequence
68820,053bf66eba3f5d9980daafd812b99161,151,7342e9ab4e268b54f9cce8cc25f11345,1,6.00,48,1.57,11.17,Foothill Vehicle Positions,t604-b2791-sl5,0.88,1
68821,053bf66eba3f5d9980daafd812b99161,173,7342e9ab4e268b54f9cce8cc25f11345,1,6.37,49,3.25,8.78,Foothill Vehicle Positions,t604-b2791-sl5,1.46,2
68822,053bf66eba3f5d9980daafd812b99161,194,7342e9ab4e268b54f9cce8cc25f11345,1,6.37,49,3.25,8.78,Foothill Vehicle Positions,t604-b2791-sl5,1.46,3
68823,053bf66eba3f5d9980daafd812b99161,200,7342e9ab4e268b54f9cce8cc25f11345,1,5.60,14,1.35,9.09,Foothill Vehicle Positions,t604-b2791-sl5,4.15,4
68824,053bf66eba3f5d9980daafd812b99161,229,7342e9ab4e268b54f9cce8cc25f11345,1,5.60,14,1.35,9.09,Foothill Vehicle Positions,t604-b2791-sl5,4.15,5
68825,053bf66eba3f5d9980daafd812b99161,245,7342e9ab4e268b54f9cce8cc25f11345,1,6.29,42,4.17,7.86,Foothill Vehicle Positions,t604-b2791-sl5,5.98,6
68826,053bf66eba3f5d9980daafd812b99161,280,7342e9ab4e268b54f9cce8cc25f11345,1,5.56,28,1.98,7.94,Foothill Vehicle Positions,t604-b2791-sl5,5.36,7
68827,053bf66eba3f5d9980daafd812b99161,287,7342e9ab4e268b54f9cce8cc25f11345,1,10.18,45,5.60,16.33,Foothill Vehicle Positions,t604-b2791-sl5,5.91,8
68828,053bf66eba3f5d9980daafd812b99161,328,7342e9ab4e268b54f9cce8cc25f11345,1,10.18,45,5.60,16.33,Foothill Vehicle Positions,t604-b2791-sl5,5.91,9
68829,053bf66eba3f5d9980daafd812b99161,355,7342e9ab4e268b54f9cce8cc25f11345,1,8.52,46,6.28,9.92,Foothill Vehicle Positions,t604-b2791-sl5,9.09,10


In [75]:
# one_route4 =  m1[m1.shape_array_key == "053bf66eba3f5d9980daafd812b99161"]

In [76]:
# one_route4.head()

In [77]:
# one_route2

In [78]:
# one_route['test_seq'] = one_route.groupby(['gtfs_dataset_key','shape_array_key', 'trip_id']).cumcount().astype(int)+1

In [79]:
# one_route

In [80]:
# one_route2['test_seq'] = one_route2.groupby(['gtfs_dataset_key','shape_array_key', 'trip_id']).cumcount().astype(int)+1

In [81]:
# one_route2.sort_values(by = ['trip_id', 'test_seq', 'stop_sequence'])# 

In [82]:
test1 = m1.melt(id_vars=[ '_gtfs_dataset_name','shape_array_key','trip_id', 'Test Stop Sequence','gtfs_dataset_key','loop_or_inlining',
                       'n_trips'], value_vars=[ 'avg_speed_mph','speed_mph','p20_speed_mph', 'p80_speed_mph'])

In [83]:
test1.shape

(11174816, 9)

In [84]:
# test1[test1.shape_array_key == "29d2bbdbeaec1d6888800f85bebf6e33"]

In [85]:
# Only need average speed/p20 speed/p80 to show up once for each stop sequence-operator-shape array
test2 = test1.drop_duplicates(subset = [ '_gtfs_dataset_name','shape_array_key','Test Stop Sequence','gtfs_dataset_key','variable','value']).reset_index(drop = True)

In [86]:
test2.shape

(4213138, 9)

### Charts  
Test with a few routes first
* Create new col that rounds up speed for plotting purposes only.

In [87]:
subset = test2[test2.shape_array_key.isin(["29d2bbdbeaec1d6888800f85bebf6e33",'754c5b012195800c38dc58e72e4f482e',
       'e3c5ed2c6fa6cd5c5cd57d46aeb3cd8e'])]

In [88]:
subset = threshold_utils.pre_clean(subset)

In [89]:
subset['Route'] = 'Operator: ' + subset['Gtfs Dataset Name'] + ' Route Type: ' + subset['Loop Or Inlining'].astype(str) + ' Shape Array: ' + subset['Shape Array Key']

In [90]:
subset = subset.rename(columns = {'Value':'Speed'})

In [91]:
subset['Speed_Int'] = subset.Speed.fillna(0).astype(int)

In [92]:
# subset['Rounded Speed'].unique()

In [93]:
def speed(row):
    # If partner is none, return Unknown.
    if row.Speed_Int == 0:
        return 0
    elif 0 < row.Speed_Int < 6:
        return 5
    elif 5 < row.Speed_Int < 11:
        return 10
    elif 10 < row.Speed_Int < 16:
        return 15
    elif 15 < row.Speed_Int < 21:
        return 20
    elif 20 < row.Speed_Int < 26:
        return 25
    elif 25 < row.Speed_Int < 31:
        return 30
    else:
        return 35

In [94]:
# Apply the function
subset["Rounded Speed"] = subset.apply(speed, axis=1)

In [95]:
# subset[['Rounded Speed', 'Speed', 'Speed_Int']]

In [96]:
subset.Variable = subset.Variable.str.title().str.replace("_"," ")

In [97]:
# One df for the actual speeds
subset_speedmph = subset[subset.Variable == 'Speed Mph'].reset_index(drop = True)

In [98]:
# One df for the percentiles
subset_other= subset[subset.Variable != 'Speed Mph'].reset_index(drop = True)

In [99]:
def alt_dropdown(df, col_for_dropdown:str, dropdown_menu_title:str):
    # Create dropdown menu
    # Exclude "none" operators which are only scheduled data
    df = df.loc[df[col_for_dropdown] != "None"][[col_for_dropdown]]
    dropdown_list = df[col_for_dropdown].unique().tolist()
    
    # Show only first operator by default
    initialize_first_op = sorted(dropdown_list)[0]
    input_dropdown = alt.binding_select(options=sorted(dropdown_list), name=dropdown_menu_title)
    
    selection = alt.selection_single(name= dropdown_menu_title,fields=[col_for_dropdown],
    bind=input_dropdown, init={col_for_dropdown: initialize_first_op})
                 
    return selection

In [100]:
selection_test = alt_dropdown(subset, "Route", "Operator/Shape Array")

In [115]:
def create_jitter_plot(df):
    
    #title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    #inline = df['Loop Or Inlining'].iloc[0]
    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q",
                    scale=alt.Scale(domain=[0, 50]),
                    title = "Speed (MPH)",
                    axis=alt.Axis(labelAngle=360, grid=False,)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Test Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="top",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="Speeds by Operator-Shape Array")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 40, 250)
    
    return chart1

In [116]:
chart1 = create_jitter_plot(subset_speedmph).add_selection(selection_test).transform_filter(selection_test)

In [117]:
chart2 = (
        alt.Chart(subset_other, width=0.5)
        .mark_circle(size=200)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", 
                    title = "Speed (MPH)",
                    
                    scale=alt.Scale(domain=[0, 50]),
                    axis=alt.Axis(labels=False, ticks = False, grid= False)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=subset_other.columns.tolist(),
            column=alt.Column(
                "Test Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    title = None,
                    titleOrient="top",
                    labelOrient="top",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
    )
    

In [118]:
chart2 = threshold_utils.chart_size(chart2, 50, 250)

In [119]:
chart2 = chart2.add_selection(selection_test).transform_filter(selection_test)

In [120]:
chart1.interactive() | chart2.interactive()

alt.HConcatChart(...)

### Draft

In [121]:
def create_jitter_plot(df):
    
    title_op = df['Gtfs Dataset Name'].iloc[0].replace('VehiclePositions','').strip()
    inline = df['Loop Or Inlining'].iloc[0]
    
    chart1 = (
        alt.Chart(df, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("Rounded Speed:Q", axis=alt.Axis(labelAngle=360)),
            color=alt.Color(
                "Variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=df.columns.tolist(),
            column=alt.Column(
                "Stop Sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"{title_op} - Route Type {inline}")
    )
    
    chart1 = threshold_utils.chart_size(chart1, 40, 250)
    
    return chart1

In [122]:
chart2 = (
        alt.Chart(anaheim_test, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

NameError: name 'anaheim_test' is not defined

In [ ]:
chart2 = threshold_utils.chart_size(chart2,80,300)

In [ ]:
chart2

In [ ]:
chart1 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "stop_sequence:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart1 = threshold_utils.chart_size(chart1,80,300)

In [ ]:
chart1

In [ ]:
def create_dot_plot2(df, col_for_dots: str, 
                    x_axis_col:str, y_axis_col:str,
                   tooltip_cols:list, chart_title:str):
  
    chart = (alt.Chart(df).mark_circle(opacity=1, size = 100).transform_window(
    id='rank()',
    groupby=[col_for_dots]).encode(
    alt.X(f'{x_axis_col}:O', sort='descending', 
          axis=alt.Axis(ticks=False, grid=True)),
    alt.Y(f'{y_axis_col}:N'), 
    color=alt.Color(f"{col_for_dots}:N", scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS), legend=None),
    tooltip = tooltip_cols)
             .properties(title = chart_title))
    
    return chart

In [ ]:
chart3 = create_dot_plot1(anaheim_test_other, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_other.columns.tolist(),  'Percentile/Average')

In [ ]:
chart3 = threshold_utils.chart_size(chart3,650,300)

In [ ]:
chart4 = create_dot_plot2(anaheim_test_speedmph, 'variable', 'stop_sequence', 'rounded_speed', anaheim_test_speedmph.columns.tolist(), 'Speed per Trip')

In [ ]:
chart4 = threshold_utils.chart_size(chart4,650,300)

In [ ]:
chart4

In [ ]:
chart3 + chart4

In [ ]:
chart7 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .configure_facet(spacing=0)
        .configure_view(stroke=None)
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart7 = threshold_utils.chart_size(chart7,80,300)

In [ ]:
chart8 = (
        alt.Chart(anaheim_test_other, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=-90,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title="Trip Duration by RT Category")
    )

In [ ]:
chart9 = (
        alt.Chart(anaheim_test_speedmph, width=0.5)
        .mark_circle(size=100)
        .encode(
            x=alt.X(
                "jitter:Q",
                title=None,
                axis=alt.Axis(values=[0], ticks=False, grid=False, labels=False),
                scale=alt.Scale(),
            ),
            y=alt.Y("rounded_speed:Q", axis=alt.Axis(labelAngle=-90)),
            color=alt.Color(
                "variable:N",
                scale=alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS),
            ),
            tooltip=anaheim_test.columns.tolist(),
            column=alt.Column(
                "stop_sequence:N",
                header=alt.Header(
                    labelAngle=360,
                    titleOrient="top",
                    labelOrient="bottom",
                    labelAlign="right",
                    labelPadding=2,
                ),
            ),
        )
        .transform_calculate(
            # Generate Gaussian jitter with a Box-Muller transform
            jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
        )
        .properties(title=f"Loop/Inling:{anaheim_test_speedmph.loop_or_inlining.iloc[0]}")
    )

In [ ]:
chart8

In [ ]:
chart9 | chart8

In [ ]:
# pip install altair==5.0.0rc3
chart5 = alt.Chart(anaheim_test_speedmph, title='Normally distributed jitter').mark_circle(size=50).encode(
    y="rounded_speed:Q",
    x="stop_sequence:N",
    yOffset="jitter:Q",
    color=alt.Color('stop_sequence:Q').legend(None)
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)


In [ ]:
chart5 = threshold_utils.chart_size(chart5,650,300)

In [ ]:
chart5